# Aggregate responses

Aggregate the synthetic person-level data to find the proportion who gave each response to each question.

## Set-up

### Packages and file paths

In [1]:
# Import required packages
from collections import defaultdict
from dataclasses import dataclass
import numpy as np
import os
import pandas as pd

In [2]:
# File paths
@dataclass(frozen=True)
class Paths:
    '''Stores paths to data and files'''
    survey = '../data/survey_data'
    synthetic_data = 'synthetic_data_raw.csv'
    aggregate = 'aggregate_responses.csv'


paths = Paths()

In [3]:
# Import functions defined elsewhere
import sys
sys.path.append('../')
from utilities.response_labels import create_response_label_dict
from create_and_process_data.functions import results_by_school_and_group

### Import raw data

In [4]:
data = pd.read_csv(os.path.join(paths.survey, paths.synthetic_data))
data.head()

,gender,transgender,sexual_orientation,neurodivergent,birth_parent1,birth_parent2,birth_you,birth_you_age,autonomy_pressure,autonomy_express,...,peer_talk_listen_lab,peer_talk_helpful_lab,peer_talk_if_lab,accept_peer_lab,year_group_lab,fsm_lab,sen_lab,ethnicity_lab,english_additional_lab,school_lab
0,4.0,2.0,6.0,NaN,2.0,1.0,2.0,1.0,2.0,5.0,...,Fully,Somewhat helpful,Very uncomfortable,Not at all,Year 10,Non-FSM,Non-SEN,Ethnic minority,No,School E
1,1.0,NaN,1.0,3.0,3.0,NaN,NaN,8.0,4.0,2.0,...,Mostly,Very helpful,Very uncomfortable,Slightly,Year 10,Non-FSM,NaN,Ethnic minority,NaN,School D
2,2.0,3.0,4.0,1.0,1.0,1.0,1.0,NaN,5.0,4.0,...,Mostly,Very helpful,Very comfortable,Not at all,Year 10,Non-FSM,Non-SEN,White British,No,School E
3,2.0,NaN,5.0,2.0,2.0,2.0,1.0,3.0,1.0,2.0,...,Fully,Somewhat helpful,Uncomfortable,Mostly,Year 10,Non-FSM,Non-SEN,White British,No,School G
4,5.0,3.0,4.0,1.0,1.0,3.0,3.0,NaN,5.0,2.0,...,Slightly,Somewhat helpful,Uncomfortable,Not at all,Year 8,Non-FSM,Non-SEN,White British,Yes,School B


## Find the proportion giving each response to each measure, within a given group

In [5]:
# Make list of columns that we want to count responses for
response_col = [col for col in data.columns if (
    col.endswith('_lab') and col not in [
        'school_lab', 'gender', 'transgender', 'sexual_orientation',
        'neurodivergent', 'birth_parent1', 'birth_parent2', 'birth_you',
        'birth_you_age', 'young_carer', 'care_experience', 'year_group', 'fsm',
        'sen', 'ethnicity', 'english_additional'])]

# Preview the list
response_col[0:10]

['gender_lab',
 'transgender_lab',
 'sexual_orientation_lab',
 'neurodivergent_lab',
 'birth_parent1_lab',
 'birth_parent2_lab',
 'birth_you_lab',
 'birth_you_age_lab',
 'autonomy_pressure_lab',
 'autonomy_express_lab']

In [6]:
# Import dictionary which contains the response options for each question,
# for which we want to know the answers to
labels = create_response_label_dict()

# Add 'NaN': 'No response' to each of the dictionaries
# They are stored as dictionary of dictionaries, so we loop through and
# update each one
for key, value in labels.items():
    value.update({np.nan: 'No response'})

# Preview one of the dictionary items
labels['autonomy']

{1: '1 - Completely not true',
 2: '2',
 3: '3',
 4: '4',
 5: '5 - Completely true',
 nan: 'No response'}

Define function for aggregating responses to each variable, designed so that it counts the occurence of all possible values, rather than basing it on only the values present (e.g. if e.g. no-one responded 3, it will just have counts of responses to 1 2 and 4, and when it comes to plotting, that can create issues).

In [7]:
def aggregate(data):
    '''
    Aggregates each of the columns provided by response_col, for the chosen
    dataset. Uses the known possible values for each column, it counts
    occurences of each (inc. number missing) and makes the answer as a single
    dataframe row, where counts and percentages and categories are stored as
    lists within cells of that row. The function returns a dataframe containing
    all of those rows.
    Inputs:
    - data - dataframe
    '''
    # Initialise list to store rows of the dataframe
    rows = list()

    # Loop through the columns of interest
    for col_lab in response_col:

        # Find the name of the numeric version of the column
        col = col_lab.replace('_lab', '')

        # Find value counts
        value_counts = data[col].value_counts(dropna=False)

        # Get all possible category values and labels from dictionary
        cat = list(labels[col].keys())
        cat_lab = list(labels[col].values())

        # Initalise list for storing counts
        counts = []
        # For each of the possible values in labels - if the value was present,
        # extract from the counts series, but if not, set count to 0
        for value in labels[col].keys():
            if value in value_counts.index:
                counts.append(value_counts[value])
            else:
                counts.append(0)

        # Convert list of counts to list of percentages, and create rounded version
        percentages = [(x/sum(counts))*100 for x in counts]

        # Create dataframe row using the calculated data
        df_row = pd.DataFrame({
            'cat': [cat],
            'cat_lab': [cat_lab],
            'count': [counts],
            'percentage': [percentages],
            'measure': col,
            'n_responses': sum(counts)
        })
        # Append to list
        rows.append(df_row)

    # Combine into a single dataframe and return
    return(pd.concat(rows))


In [8]:
# Create version where every question has count 0, to use when a school has no
# pupils of a particular subgroup (i.e. no-one in certain FSM/SEN/gender/year)
no_pupils = aggregate(data)
no_pupils[['count', 'percentage', 'n_responses']] = 0
no_pupils.head()

,cat,cat_lab,count,percentage,measure,n_responses
0,"[1, 2, 3, 4, 5, 6, nan]","[Girl, Boy, Non-binary, I describe myself in a...",0,0,gender,0
0,"[1, 2, 3, 4, 5, nan]","[Yes, No, Prefer not to say, I describe myself...",0,0,transgender,0
0,"[1, 2, 3, 4, 5, 6, nan]","[Bi/pansexual, Gay/lesbian, Heterosexual/strai...",0,0,sexual_orientation,0
0,"[1, 2, 3, nan]","[Yes, No, Unsure, No response]",0,0,neurodivergent,0
0,"[1, 2, 3, nan]","[Yes, No, I don't know, No response]",0,0,birth_parent1,0


In [9]:
# Find results of aggregation for each school and pupil group
result = results_by_school_and_group(
    data=data, agg_func=aggregate, no_pupils=no_pupils)

# Hide results where n<10
result.loc[result['n_responses'] < 10,
           ['count', 'percentage', 'n_responses']] = np.nan

# Preview head of dataframe
result.head()

,cat,cat_lab,count,percentage,measure,n_responses,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab
0,"[1, 2, 3, 4, 5, 6, nan]","[Girl, Boy, Non-binary, I describe myself in a...","[16, 28, 19, 17, 13, 19, 16]","[12.5, 21.875, 14.84375, 13.28125, 10.15625, 1...",gender,128.0,School A,All,All,All,All
0,"[1, 2, 3, 4, 5, nan]","[Yes, No, Prefer not to say, I describe myself...","[23, 23, 16, 28, 23, 15]","[17.96875, 17.96875, 12.5, 21.875, 17.96875, 1...",transgender,128.0,School A,All,All,All,All
0,"[1, 2, 3, 4, 5, 6, nan]","[Bi/pansexual, Gay/lesbian, Heterosexual/strai...","[22, 25, 15, 20, 28, 15, 3]","[17.1875, 19.53125, 11.71875, 15.625, 21.875, ...",sexual_orientation,128.0,School A,All,All,All,All
0,"[1, 2, 3, nan]","[Yes, No, Unsure, No response]","[50, 35, 35, 8]","[39.0625, 27.34375, 27.34375, 6.25]",neurodivergent,128.0,School A,All,All,All,All
0,"[1, 2, 3, nan]","[Yes, No, I don't know, No response]","[39, 43, 40, 6]","[30.46875, 33.59375, 31.25, 4.6875]",birth_parent1,128.0,School A,All,All,All,All


In [10]:
# Preview school where I set there to be no SEN pupils
# Should show as NaN, as count was 0 which is <10
result[(result['school_lab'] == 'School B') & (result['sen_lab'] == 'SEN')]

,cat,cat_lab,count,percentage,measure,n_responses,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab
0,"[1, 2, 3, 4, 5, 6, nan]","[Girl, Boy, Non-binary, I describe myself in a...",NaN,NaN,gender,NaN,School B,All,All,All,SEN
0,"[1, 2, 3, 4, 5, nan]","[Yes, No, Prefer not to say, I describe myself...",NaN,NaN,transgender,NaN,School B,All,All,All,SEN
0,"[1, 2, 3, 4, 5, 6, nan]","[Bi/pansexual, Gay/lesbian, Heterosexual/strai...",NaN,NaN,sexual_orientation,NaN,School B,All,All,All,SEN
0,"[1, 2, 3, nan]","[Yes, No, Unsure, No response]",NaN,NaN,neurodivergent,NaN,School B,All,All,All,SEN
0,"[1, 2, 3, nan]","[Yes, No, I don't know, No response]",NaN,NaN,birth_parent1,NaN,School B,All,All,All,SEN
...,...,...,...,...,...,...,...,...,...,...,...
0,"[8, 10, nan]","[Year 8, Year 10, No response]",NaN,NaN,year_group,NaN,School B,All,All,All,SEN
0,"[0, 1, nan]","[Non-FSM, FSM, No response]",NaN,NaN,fsm,NaN,School B,All,All,All,SEN
0,"[0, 1, nan]","[Non-SEN, SEN, No response]",NaN,NaN,sen,NaN,School B,All,All,All,SEN
0,"[1, 2, nan]","[Ethnic minority, White British, No response]",NaN,NaN,ethnicity,NaN,School B,All,All,All,SEN


## Add groups for each measure

This uses add_keys() like `1_create_synthetic_data.ipynb`.

In [11]:
# Initialise dictionary of groups
groups = defaultdict(str)

# Define function for adding multiple keys with the same value
def add_keys(value, keys):
    '''
    Add multiple keys with the same value to the dictionary
    Inputs:
    value: String which is the value for all the keys
    keys: Array with the keys
    '''
    groups.update(dict.fromkeys(keys, value))

# Add the groups to the dictionary
add_keys('demographic', [
    'gender',
    'transgender',
    'sexual_orientation',
    'neurodivergent',
    'birth_parent1',
    'birth_parent2',
    'birth_you',
    'birth_you_age',
    'young_carer',
    'care_experience',
    'year_group',
    'fsm',
    'sen',
    'ethnicity',
    'english_additional'])

add_keys('autonomy', [
    'autonomy_pressure',
    'autonomy_express',
    'autonomy_decide',
    'autonomy_told',
    'autonomy_myself',
    'autonomy_choice'])
add_keys('life_satisfaction', ['life_satisfaction'])
add_keys('optimism', [
    'optimism_future',
    'optimism_best',
    'optimism_good',
    'optimism_work'])
add_keys('wellbeing', [
    'wellbeing_optimistic',
    'wellbeing_useful',
    'wellbeing_relaxed',
    'wellbeing_problems',
    'wellbeing_thinking',
    'wellbeing_close',
    'wellbeing_mind'])
add_keys('esteem', [
    'esteem_satisfied',
    'esteem_qualities',
    'esteem_well',
    'esteem_value',
    'esteem_good'])
add_keys('stress', [
    'stress_control',
    'stress_overcome',
    'stress_confident',
    'stress_way'])
add_keys('appearance', ['appearance_happy', 'appearance_feel'])
add_keys('negative', [
    'negative_lonely',
    'negative_unhappy',
    'negative_like',
    'negative_cry',
    'negative_school',
    'negative_worry',
    'negative_sleep',
    'negative_wake',
    'negative_shy',
    'negative_scared'])
add_keys('lonely', ['lonely'])
add_keys('support', ['support_ways', 'support_look'])

add_keys('sleep', ['sleep'])
add_keys('physical', ['physical_days', 'physical_hours'])
add_keys('free_like', ['free_like'])
add_keys('media', ['media_hours'])
add_keys('places', [
    'places_freq',
    'places_barriers___1',
    'places_barriers___2',
    'places_barriers___3',
    'places_barriers___4',
    'places_barriers___5',
    'places_barriers___6',
    'places_barriers___7',
    'places_barriers___8',
    'places_barriers___9'])
add_keys('school_belong', ['school_belong'])
add_keys('staff_relationship', [
    'staff_interest', 'staff_believe', 'staff_best', 'staff_listen'])

add_keys('talk', [
    'staff_talk', 'staff_talk_listen', 'staff_talk_helpful', 'staff_talk_if',
    'home_talk', 'home_talk_listen', 'home_talk_helpful', 'home_talk_if',
    'peer_talk', 'peer_talk_listen', 'peer_talk_helpful', 'peer_talk_if'])
add_keys('accept', [
    'accept_staff', 'accept_home', 'accept_local', 'accept_peer'])

add_keys('home_relationship', [
    'home_interest', 'home_believe', 'home_best', 'home_listen'])
add_keys('home_happy', ['home_happy'])

add_keys('local_env', ['local_safe', 'local_support', 'local_trust',
                       'local_neighbours', 'local_places'])
add_keys('discrim', ['discrim_race', 'discrim_gender', 'discrim_orientation',
                     'discrim_disability', 'discrim_faith'])
add_keys('belong_local', ['belong_local'])
add_keys('wealth', ['wealth'])
add_keys('future', ['future_options', 'future_interest', 'future_support'])
add_keys('climate', ['climate'])
add_keys('social', [
    'social_along', 'social_time', 'social_support', 'social_hard'])
add_keys('bully', ['bully_physical', 'bully_other', 'bully_cyber'])

# Preview the dictionary
groups

defaultdict(str,
            {'gender': 'demographic',
             'transgender': 'demographic',
             'sexual_orientation': 'demographic',
             'neurodivergent': 'demographic',
             'birth_parent1': 'demographic',
             'birth_parent2': 'demographic',
             'birth_you': 'demographic',
             'birth_you_age': 'demographic',
             'young_carer': 'demographic',
             'care_experience': 'demographic',
             'year_group': 'demographic',
             'fsm': 'demographic',
             'sen': 'demographic',
             'ethnicity': 'demographic',
             'english_additional': 'demographic',
             'autonomy_pressure': 'autonomy',
             'autonomy_express': 'autonomy',
             'autonomy_decide': 'autonomy',
             'autonomy_told': 'autonomy',
             'autonomy_myself': 'autonomy',
             'autonomy_choice': 'autonomy',
             'life_satisfaction': 'life_satisfaction',
             'opt

In [12]:
# Add groups and preview
result['group'] = result['measure'].map(groups)
result.head()

,cat,cat_lab,count,percentage,measure,n_responses,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab,group
0,"[1, 2, 3, 4, 5, 6, nan]","[Girl, Boy, Non-binary, I describe myself in a...","[16, 28, 19, 17, 13, 19, 16]","[12.5, 21.875, 14.84375, 13.28125, 10.15625, 1...",gender,128.0,School A,All,All,All,All,demographic
0,"[1, 2, 3, 4, 5, nan]","[Yes, No, Prefer not to say, I describe myself...","[23, 23, 16, 28, 23, 15]","[17.96875, 17.96875, 12.5, 21.875, 17.96875, 1...",transgender,128.0,School A,All,All,All,All,demographic
0,"[1, 2, 3, 4, 5, 6, nan]","[Bi/pansexual, Gay/lesbian, Heterosexual/strai...","[22, 25, 15, 20, 28, 15, 3]","[17.1875, 19.53125, 11.71875, 15.625, 21.875, ...",sexual_orientation,128.0,School A,All,All,All,All,demographic
0,"[1, 2, 3, nan]","[Yes, No, Unsure, No response]","[50, 35, 35, 8]","[39.0625, 27.34375, 27.34375, 6.25]",neurodivergent,128.0,School A,All,All,All,All,demographic
0,"[1, 2, 3, nan]","[Yes, No, I don't know, No response]","[39, 43, 40, 6]","[30.46875, 33.59375, 31.25, 4.6875]",birth_parent1,128.0,School A,All,All,All,All,demographic


## Add labels for each measure

In [13]:
# Define labels
labels = {
    'gender': 'Gender',
    'transgender': 'Do you consider yourself to be transgender?',
    'sexual_orientation': 'Sexual orientation',
    'neurodivergent': 'Do you identify as neurodivergent?',
    'autonomy_pressure': 'I feel pressured in my life',
    'autonomy_express': 'I generally feel free to express my ideas and opinions',
    'autonomy_decide': 'I feel like I am free to decide for myself how to live my life',
    'autonomy_told': 'In my daily life I often have to do what I am told',
    'autonomy_myself': 'I feel I can pretty much be myself in daily situations',
    'autonomy_choice': 'I have enough choice about how I spend my time',
    'life_satisfaction': 'Overall, how satisfied are you with your life nowadays?',
    'optimism_future': 'I am optimistic about my future',
    'optimism_best': 'In uncertain times, I expect the best',
    'optimism_good': 'I think good things are going to happen to me',
    'optimism_work': 'I believe that things will work out, no matter how difficult they seem',
    'wellbeing_optimistic': '''I've been feeling optimistic about the future''',
    'wellbeing_useful': '''I've been feeling useful''',
    'wellbeing_relaxed': '''I've been feeling relaxed''',
    'wellbeing_problems': '''I've been dealing with problems well''',
    'wellbeing_thinking': '''I've been thinking clearly''',
    'wellbeing_close': '''I've been feeling close to other people''',
    'wellbeing_mind': '''I've been able to make up my own mind about things''',
    'esteem_satisfied': 'On the whole, I am satisfied with myself',
    'esteem_qualities': 'I feel that I have a number of good qualities',
    'esteem_well': 'I am able to do things as well as most other people',
    'esteem_value': 'I am a person of value',
    'esteem_good': 'I feel good about myself',
    'stress_control': 'Felt you were unable to control the important things in your life',
    'stress_overcome': 'Felt that difficulties were piling up so high that you could not overcome them',
    'stress_confident': 'Felt confident about your ability to handle your personal problems',
    'stress_way': 'Felt that things were going your way',
    'appearance_happy': 'How happy are you with your appearance (the way that you look)?',
    'appearance_feel': 'My appearance affects how I feel about myself',
    'negative_lonely': 'I feel lonely',
    'negative_unhappy': 'I am unhappy',
    'negative_like': 'Nobody likes me',
    'negative_cry': 'I cry a lot',
    'negative_school': 'I worry when I am at school',
    'negative_worry': 'I worry a lot',
    'negative_sleep': 'I have problems sleeping',
    'negative_wake': 'I wake up in the night',
    'negative_shy': 'I am shy',
    'negative_scared': 'I feel scared',
    'lonely': 'How often do you feel lonely?',
    'support_ways': 'I have ways to support myself (e.g. to cope, or help myself feel better)',
    'support_look': 'I know where to look for advice on how to support myself',
    'sleep': 'Is the amount of sleep you normally get enough for you to feel awake and concentrate on your school work during the day?',
    'physical_days': 'How many days in a usual week are you physically active?',
    'physical_hours': 'How long on average do you spend being physically active?',
    'free_like': 'How often can you do things that you like in your free time?',
    'media_hours': 'On a normal weekday during term time, how much time do you spend on social media?',
    'places_freq': 'How many activities/places are there in your local area, that you choose to or would want to go to in your free time?',
    'places_barriers___1': '''There's nothing to do''',
    'places_barriers___2': '''I'm unable to get there and back''',
    'places_barriers___3': '''It's too expensive (to get there or take part)''',
    'places_barriers___4': 'Poor weather',
    'places_barriers___5': 'I have no-one to go with',
    'places_barriers___6': '''It's too busy''',
    'places_barriers___7': 'I feel uncomfortable/anxious about other people who might be there',
    'places_barriers___8': '''My parents/carers don't allow me to go''',
    'places_barriers___9': 'Other',
    'school_belong': 'I feel that I belong/belonged at my school',
    'staff_interest': 'At school there is an adult who... is interested in my schoolwork',
    'staff_believe': 'At school there is an adult who... believes that I will be a success',
    'staff_best': 'At school there is an adult who... wants me to do my best',
    'staff_listen': 'At school there is an adult who... listens to me when I have something to say',
    'staff_talk': 'Talked about feeling down with... an adult at school',
    'staff_talk_listen': 'Did you feel listened to when you spoke with... an adult at school',
    'staff_talk_helpful': 'Did you receive advice that you found helpful from... an adult at school',
    'staff_talk_if': 'How would you feel about speaking with... an adult at school',
    'accept_staff': 'Adults at your school',
    'home_interest': 'At home there is an adult who... is interested in my schoolwork',
    'home_believe': 'At home there is an adult who... believes that I will be a success',
    'home_best': 'At home there is an adult who... wants me to do my best',
    'home_listen': 'At home there is an adult who... listens to me when I have something to say',
    'home_talk': 'Talked about feeling down with... one of your parents/carers',
    'home_talk_listen': 'Did you feel listened to when you spoke with... one of your parents/carers',
    'home_talk_helpful': 'Did you receive advice that you found helpful from... one of your parents/carers',
    'home_talk_if': 'How would you feel about speaking with... one of your parents/carers',
    'accept_home': 'Your parents/carers',
    'home_happy': 'How happy are you with the home that you live in?',
    'young_carer': 'In the last year, have you regularly taken on caring responsibilities for a family member - e.g. due to illness, disability, mental health condition or drug/alcohol dependency?',
    'care_experience': 'Are you or have you ever been in care (living in a foster placement, residential placement, or private/kinship care)?',
    'local_safe': 'How safe do you feel when in your local area?',
    'local_support': 'People around here support each other with their wellbeing',
    'local_trust': 'You can trust people around here',
    'local_neighbours': 'I could ask for help or a favour from neighbours',
    'local_places': 'There are good places to spend your free time (e.g., leisure centres, parks, shops)',
    'discrim_race': 'How often do people make you feel bad because of... your race, skin colour or where you were born?',
    'discrim_gender': 'How often do people make you feel bad because of... your gender?',
    'discrim_orientation': 'How often do people make you feel bad because of... your sexual orientation?',
    'discrim_disability': 'How often do people make you feel bad because of... disability?',
    'discrim_faith': 'How often do people make you feel bad because of... your religion/faith?',
    'belong_local': 'I feel like I belong in my local area',
    'accept_local': 'People in your local area',
    'wealth': 'Compared to your friends, is your family richer, poorer or about the same?',
    'future_options': 'How many options are available?',
    'future_interest': 'How do you feel about the options available?',
    'future_support': 'Do you feel (or think you would feel) supported to explore options that interest you, even if no-one else around you has done them before?',
    'climate': 'How often do you worry about the impact of climate change on your future?',
    'social_along': 'I get along with people around me',
    'social_time': 'People like to spend time with me',
    'social_support': 'I feel supported by my friends',
    'social_hard': 'My friends care about me when times are hard (for example if I am sick or have done something wrong)',
    'bully_physical': 'How often do you get physically bullied at school? By this we mean getting hit, pushed around, threatened, or having belongings stolen.',
    'bully_other': 'How often do you get bullied in other ways at school? By this we mean insults, slurs, name calling, threats, getting left out or excluded by others, or having rumours spread about you on purpose.',
    'bully_cyber': 'How often do you get cyber-bullied? By this we mean someone sending mean text or online messages about you, creating a website making fun of you, posting pictures that make you look bad online, or sharing them with others.',
    'peer_talk': 'Talked about feeling down with... another person your age',
    'peer_talk_listen': 'Did you feel listened to when you spoke with... another person your age',
    'peer_talk_helpful': 'Did you receive advice that you found helpful from... another person your age',
    'peer_talk_if': 'How would you feel about speaking with... another person your age',
    'accept_peer': 'Other people your age',
    'birth_parent1': 'Was birth parent 1 born outside the UK?',
    'birth_parent2': 'Was birth parent 2 born outside the UK?',
    'birth_you': 'Were you born outside the UK?',
    'birth_you_age': 'How old were you when you arrived in the UK?',
    'year_group': 'Year group',
    'fsm': 'Free school meals',
    'sen': 'Special educational needs',
    'ethnicity': 'Ethnicity',
    'english_additional': 'English as an additional lanaguage'
}

In [14]:
# Add labels
result['measure_lab'] = result['measure'].map(labels)

# Preview
result.head()

,cat,cat_lab,count,percentage,measure,n_responses,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab,group,measure_lab
0,"[1, 2, 3, 4, 5, 6, nan]","[Girl, Boy, Non-binary, I describe myself in a...","[16, 28, 19, 17, 13, 19, 16]","[12.5, 21.875, 14.84375, 13.28125, 10.15625, 1...",gender,128.0,School A,All,All,All,All,demographic,Gender
0,"[1, 2, 3, 4, 5, nan]","[Yes, No, Prefer not to say, I describe myself...","[23, 23, 16, 28, 23, 15]","[17.96875, 17.96875, 12.5, 21.875, 17.96875, 1...",transgender,128.0,School A,All,All,All,All,demographic,Do you consider yourself to be transgender?
0,"[1, 2, 3, 4, 5, 6, nan]","[Bi/pansexual, Gay/lesbian, Heterosexual/strai...","[22, 25, 15, 20, 28, 15, 3]","[17.1875, 19.53125, 11.71875, 15.625, 21.875, ...",sexual_orientation,128.0,School A,All,All,All,All,demographic,Sexual orientation
0,"[1, 2, 3, nan]","[Yes, No, Unsure, No response]","[50, 35, 35, 8]","[39.0625, 27.34375, 27.34375, 6.25]",neurodivergent,128.0,School A,All,All,All,All,demographic,Do you identify as neurodivergent?
0,"[1, 2, 3, nan]","[Yes, No, I don't know, No response]","[39, 43, 40, 6]","[30.46875, 33.59375, 31.25, 4.6875]",birth_parent1,128.0,School A,All,All,All,All,demographic,Was birth parent 1 born outside the UK?


## Save to csv

In [15]:
result.to_csv(os.path.join(paths.survey, paths.aggregate), index=False)